# Set up client tools for surrogate model

In [ ]:
from lume_epics.client.widgets.plots import ImagePlot
from lume_epics.client.widgets.sliders import build_sliders
from lume_epics.client.controller import Controller
from lume_model.utils import load_variables

from bokeh.io import output_notebook, show
from bokeh import palettes
from bokeh.layouts import column, row
from bokeh.models import LinearColorMapper
# load bokeh
output_notebook()

# Load the model variables

In [ ]:
prefix = "test"
variable_filename =  "surrogate_model_variables.pickle"
input_variables, output_variables = load_variables(variable_filename)

# Build sliders and image plot

In [ ]:
 # build sliders for the command process variable database
inputs = [input_variables["phi(1)"], 
          input_variables["maxb(2)"], 
          input_variables["laser_radius"],
          input_variables["total_charge:value"],
         ]
controller = Controller("ca") # can also use channel access
sliders = build_sliders(inputs, controller, prefix)

# create image plot
output_variables = [output_variables["x:y"]]
image_plot = ImagePlot(output_variables, controller, prefix)

pal = palettes.viridis(256)
#color_mapper = LinearColorMapper(palette=pal, low=0, high=256)

image_plot.build_plot(palette=pal)

# Set up image update callback
def image_update_callback():
    image_plot.update()

# Render the application

In [ ]:
# function for rendering the application in the bokeh server
def render_app(doc):
    doc.title = "Demo App"
    doc.add_root(
        column(
            row(column(sliders, width=350), column(image_plot.plot, width=350)), 
        )
    )
    doc.add_periodic_callback(image_update_callback, 250)

    
show(render_app)

# NOTE: x-min, x-max, y-min, y-max are updated directly in the variable. The server then handles serving of the metadata by protocol